In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
!pip install import_ipynb 
!conda install -c conda-forge import_ipynb

4490.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


4497.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/Users/gabesmithline/miniconda3/lib/python3.10/site-packages/conda/base/context.py:200: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(
Channels:
 - conda-forge
 - defaults
 - pytorch
Platform: osx-arm64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - import_ipynb

Current channels:

  - https://conda.anaconda.org/conda-forge
  - defaults
  - https://conda.anaconda.org/pytorch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at

In [52]:
import sys

sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field
from math import prod, sqrt
sys.path.append('../caif_negotiation/')

# Import the notebook
#import import_ipynb
#%run '../test_game_eval.ipynb'  
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from prompts.make_prompt_bargain import make_prompt_bargain
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)
import concurrent.futures


pathology_results = pd.DataFrame()  
import itertools
envy_results_history = {}
from eval.metrics import *
from utils.helpers import *
from utils.negotiation_game import *
from eval.game_data import *
import json
import pickle
import jsonpickle



In [53]:
import time
import pandas as pd
import torch
import numpy as np
from math import sqrt, prod

# ------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------
#prompt_style = 'llama_3.3_70b_test_new_data_object'
#prompt_style = 'gemini_2.0_test_new_data_object_pickle'
prompt_style = 'openai_4o_test_new_data_object'
llm_type = 'openai'
date = '1_28_2025'
max_rounds = 3
games = 1
circles = [5]



In [54]:

def calculate_discounted_value(offer, values, gamma, round_num):
    if offer is None:
        return 0
    base_value = sum(v * q for v, q in zip(values, offer))
    return base_value * (gamma ** (round_num - 1))

# ------------------------------------------------------------------------
# Helper function: Discounted value
# ------------------------------------------------------------------------
def calculate_discounted_value(offer, values, gamma, realization_round):
    """
    Returns the discounted value of `offer` for an agent with utility `values`,
    discount factor `gamma`, and the realization round number.
    """
    if offer is None:
        return 0
    base_value = sum(v * q for v, q in zip(values, offer))
    return base_value * (gamma ** (realization_round - 1))

# ------------------------------------------------------------------------
# Helper function: Detect pathology #4 (accepting worse than outside)
# ------------------------------------------------------------------------
def check_accepting_worse_than_outside(current_player, p1_kept, p2_kept, game):
    """
    PATHOLOGY 4: Accepting an offer worse than your outside offer.
    If the current player accepted, check if the portion they get
    is less than their outside offer.
    """
    accepting_worse = False
    action = game.players[current_player - 1].action
    if action == "ACCEPT":
        if current_player == 1 and p1_kept is not None:
            if np.dot(game.player_values[0], p1_kept) < game.outside_offer_values[0]:
                accepting_worse = True
        elif current_player == 2 and p2_kept is not None:
            if np.dot(game.player_values[1], p2_kept) < game.outside_offer_values[1]:
                accepting_worse = True
    return accepting_worse

# ------------------------------------------------------------------------
# Helper function: Detect pathology #5 (walking away from a better offer)
# ------------------------------------------------------------------------
def check_walking_away_from_better(current_player, p1_kept, p2_kept, game):
    """
    PATHOLOGY 5: Walking away from an offer better than your outside offer.
    """
    walking_away_better = False
    action = game.players[current_player - 1].action
    if ("WALK" in action) or (
        current_player == 2
        and action == "COUNTEROFFER"
        and game.current_round == game.max_rounds
    ):
        if current_player == 1 and p1_kept is not None:
            if np.dot(game.player_values[0], p1_kept) > game.outside_offer_values[0]:
                walking_away_better = True
        elif current_player == 2 and p2_kept is not None:
            if np.dot(game.player_values[1], p2_kept) > game.outside_offer_values[1]:
                walking_away_better = True
    return walking_away_better

# ------------------------------------------------------------------------
# Helper function: Determine validity of a WALK
# ------------------------------------------------------------------------
def determine_walk_away_type(current_player, game):
    """
    Checks if the current player's action is 'INVALID WALK' or 'WALK'
    and returns an appropriate walk_away_type. Otherwise returns None.
    """
    action = game.players[current_player - 1].action
    if "INVALID WALK" in action:
        return "INVALID"
    elif "WALK" in action:
        return "VALID"
    return None

# ------------------------------------------------------------------------
# Helper function: Update who-keeps-what (p1_kept, p2_kept)
# ------------------------------------------------------------------------
def update_kept_portions(current_player, game, p1_kept, p2_kept):
    """
    If there's a new COUNTEROFFER from the current player, update
    p1_kept and p2_kept accordingly.
    """
    action = game.players[current_player - 1].action
    if action == "COUNTEROFFER":
        if current_player == 1:
            # P1 is proposing, so P1's kept portion is whatever is left
            # and P2 is offered game.current_offer.offer
            p1_kept = game.items - np.array(game.current_offer.offer)
            p2_kept = np.array(game.current_offer.offer)
        else:  # current_player == 2
            # P2 is proposing, so P2's kept portion is whatever is left
            # and P1 is offered game.current_offer.offer
            p1_kept = np.array(game.current_offer.offer)
            p2_kept = game.items - np.array(game.current_offer.offer)
    return p1_kept, p2_kept

# ------------------------------------------------------------------------
# Helper function: Final round resolution
# ------------------------------------------------------------------------
def handle_final_round(
    game_num,
    current_round,
    current_player,
    game,
    prev_offer,
    p1_kept,
    p2_kept,
    p1_values,
    p2_values,
    p1_offers,
    accepting_an_offer_worse_than_outside_offer,
    pareto_front
):
    """
    Handle the final round's action, compute final metrics, and prepare data for recording.

    Args:
        game_num (int): The current game number.
        current_round (int): The current round number.
        current_player (int): The current player's number (1 or 2).
        game (NegotiationGame): The game instance.
        prev_offer (Offer): The previous offer made in the game.
        p1_kept (list): Player 1's kept allocation.
        p2_kept (list): Player 2's kept allocation.
        p1_values (list): List of Player 1's values across rounds.
        p2_values (list): List of Player 2's values across rounds.
        p1_offers (list): List of Player 1's offers.
        accepting_an_offer_worse_than_outside_offer (bool): Flag for pathology #4.
        pareto_front (list): The Pareto frontier for reference.

    Returns:
        dict: A dictionary containing all metrics to be recorded for the final step.
    """
    # Initialize metrics dictionary
    metrics = {
        "game_num": game_num,
        "step_num": 3.5,  # Assign a unique step number for the final round
        "round_num": current_round,
        "player": current_player,
        "action_played": None,
        "discount_rate": game.gamma ** (current_round - 1),
        "offer": list(game.current_offer.offer) if game.current_offer else [],
        "value": None,
        "undiscounted_value": None,
        "p1_valuations": list(game.player_values[0]),
        "p2_valuations": list(game.player_values[1]),
        "p1_kept_allocation": None,
        "p2_kept_allocation": None,
        "p1_final_value": None,
        "p2_final_value": None,
        "items": list(game.items),
        "your_side_of_current_offer": None,  # Adjust if applicable
        "outside_offer": None,  # Final round may not have outside_offer
        "outside_offer_undiscounted": None,  # Final round outside offers already considered
        "accepting_an_offer_worse_than_outside_offer": False,
        "making_an_offer_worse_for_you_than_your_outside_offer": False,  # Final round handled separately
        "walking_away_from_an_offer_better_than_your_outside_offer": False,  # Final round handled separately
        "offer_no_items_or_all_items": False,  # Final round handled separately
        "making_offer_worse_than_previous": False,  # Final round handled separately
        "nash_welfare": None,
        "proposal_proportion_player_1": None,
        "proposal_proportion_player_2": None,
        "concession_size": None,  # Final round no concessions
        "security_level_player_1": 0.0,
        "security_level_player_2": 0.0,
        "average_concession_size": None,  # To be computed post-game if needed
        "rawlsian_welfare": None,
        "gini_coefficient": None,
        "utilitarian_welfare": None,
        "jain_fairness_index": None,
        "on_pareto_frontier": False,
        "mean_absolute_difference": None,
        "walk_type": None
    }

    # Determine the final action and compute final values
    if game.current_offer and game.current_offer != prev_offer:
        # Player 2 made a final COUNTEROFFER
        print(f"\nPlayer {current_player}'s final action: COUNTEROFFER {game.current_offer.offer}")
        p1_value = game.outside_offer_values[0] * (game.gamma ** (current_round - 1))
        p2_value = game.outside_offer_values[1] * (game.gamma ** (current_round - 1))
        print("\nGame ended after max rounds - both players get outside offers")

        # Assign values
        metrics["action_played"] = "COUNTEROFFER"
        metrics["value"] = p1_value if current_player == 1 else p2_value
        metrics["undiscounted_value"] = (
            p1_value / (game.gamma ** (current_round - 1)) if current_player == 1 else
            p2_value / (game.gamma ** (current_round - 1))
        )
        metrics["p1_final_value"] = p1_value
        metrics["p2_final_value"] = p2_value

        # No allocations kept
        p1_kept = [0] * game.num_items
        p2_kept = [0] * game.num_items
        metrics["p1_kept_allocation"] = p1_kept
        metrics["p2_kept_allocation"] = p2_kept

    elif game.current_offer == prev_offer:
        # Player 2 ACCEPTED the final offer
        print("\nPlayer {current_player}'s final action: ACCEPT")
        # Player 2 accepted Player 1's final offer
        p1_kept = game.items - np.array(game.current_offer.offer)
        p1_value = calculate_discounted_value(
            p1_kept, game.player_values[0], game.gamma, current_round
        )
        p2_value = calculate_discounted_value(
            game.current_offer.offer, game.player_values[1], game.gamma, current_round
        )
        print(f"\nRound {current_round} Final Values:")
        print(f"Player 1: {p1_value:.2f}")
        print(f"Player 2: {p2_value:.2f}")

        # Assign values
        metrics["action_played"] = "ACCEPT"
        metrics["value"] = p1_value if current_player == 1 else p2_value
        metrics["undiscounted_value"] = (
            p1_value / (game.gamma ** (current_round - 1)) if current_player == 1 else
            p2_value / (game.gamma ** (current_round - 1))
        )
        metrics["p1_final_value"] = p1_value
        metrics["p2_final_value"] = p2_value

        # Assign allocations
        metrics["p1_kept_allocation"] = list(p1_kept)
        metrics["p2_kept_allocation"] = list(game.current_offer.offer)

        # Check pathology #4
        if game.outside_offer_values[1] > np.dot(game.player_values[1], game.current_offer.offer):
            accepting_an_offer_worse_than_outside_offer = True
            metrics["accepting_an_offer_worse_than_outside_offer"] = True

    else:
        # Player 2 WALKED AWAY
        print("\nPlayer {current_player}'s final action: WALK")
        p1_value = game.outside_offer_values[0] * (game.gamma ** (current_round - 1))
        p2_value = game.outside_offer_values[1] * (game.gamma ** (current_round - 1))
        print("\nGame ended after max rounds - both players get outside offers")

        # Assign values
        metrics["action_played"] = "WALK"
        metrics["value"] = None  # No specific value since walked away
        metrics["undiscounted_value"] = None
        metrics["p1_final_value"] = p1_value
        metrics["p2_final_value"] = p2_value

        # No allocations kept
        p1_kept = [0] * game.num_items
        p2_kept = [0] * game.num_items
        metrics["p1_kept_allocation"] = p1_kept
        metrics["p2_kept_allocation"] = p2_kept

    # Compute additional metrics only if action is ACCEPT or COUNTEROFFER
    if metrics["action_played"] in ("ACCEPT", "COUNTEROFFER"):
        # Compute Nash Welfare
        nash_welfare = sqrt(prod([
            p1_value,
            p2_value
        ]))
        metrics["nash_welfare"] = nash_welfare

        # Compute Utilitarian Welfare
        utilitarian_welfare = p1_value + p2_value
        metrics["utilitarian_welfare"] = utilitarian_welfare

        # Compute Rawlsian Welfare
        rawlsian_welfare = min(p1_value, p2_value)
        metrics["rawlsian_welfare"] = rawlsian_welfare

        # Compute Gini Coefficient
        if utilitarian_welfare > 0:
            gini_coefficient = abs(p1_value - p2_value) / (4.0 * utilitarian_welfare)
        else:
            gini_coefficient = 0.0
        metrics["gini_coefficient"] = gini_coefficient

        # Compute Mean Absolute Difference
        if p1_value == 0.0 and p2_value == 0.0:
            mean_absolute_difference = 0.0
        else:
            mean_absolute_difference = abs(p1_value - p2_value) / 2.0
        metrics["mean_absolute_difference"] = mean_absolute_difference

        # Compute Jain's Fairness Index
        if utilitarian_welfare > 0:
            mean_utility = utilitarian_welfare / 2.0
            variance = (p1_value**2 + p2_value**2) / 2.0 - mean_utility**2
            variance = max(variance, 0.0)  # Correct for negative variance due to precision
            coefficient_of_variation = (
                np.sqrt(variance) / mean_utility if mean_utility != 0 else 0.0
            )
            jain_fairness_index = 1 / (1 + coefficient_of_variation ** 2)
        else:
            jain_fairness_index = 0.0
        metrics["jain_fairness_index"] = jain_fairness_index

        # Compute Security Levels
        security_level_player_1 = max(0.0, game.outside_offer_values[0] - p1_value)
        security_level_player_2 = max(0.0, game.outside_offer_values[1] - p2_value)
        metrics["security_level_player_1"] = security_level_player_1
        metrics["security_level_player_2"] = security_level_player_2

        # Determine walk_type based on final action
        walk_type = None
        if metrics["action_played"] == "WALK":
            walk_type = "Player2_WALK"
        elif metrics["action_played"] == "COUNTEROFFER":
            walk_type = "Final_COUNTEROFFER"
        elif metrics["action_played"] == "ACCEPT":
            walk_type = "Final_ACCEPT"
        metrics["walk_type"] = walk_type

        # Check if on Pareto Frontier
        on_pareto_frontier = False
        for vals in pareto_front:
            if vals["type"] == "outside_offer" and game.current_offer is None:
                on_pareto_frontier = True
                break
            elif vals["type"] == "allocation":
                if (np.array_equal(vals["agent1"], p1_kept) and
                        np.array_equal(vals["agent2"], p2_kept)):
                    on_pareto_frontier = True
                    break
        metrics["on_pareto_frontier"] = on_pareto_frontier

    else:
        # For WALK action, compute welfare metrics based on outside offers
        nash_welfare = sqrt(prod([
            p1_value,
            p2_value
        ]))
        metrics["nash_welfare"] = nash_welfare

        utilitarian_welfare = p1_value + p2_value
        metrics["utilitarian_welfare"] = utilitarian_welfare

        rawlsian_welfare = min(p1_value, p2_value)
        metrics["rawlsian_welfare"] = rawlsian_welfare

        if utilitarian_welfare > 0:
            gini_coefficient = abs(p1_value - p2_value) / (4.0 * utilitarian_welfare)
        else:
            gini_coefficient = 0.0
        metrics["gini_coefficient"] = gini_coefficient

        if p1_value == 0.0 and p2_value == 0.0:
            mean_absolute_difference = 0.0
        else:
            mean_absolute_difference = abs(p1_value - p2_value) / 2.0
        metrics["mean_absolute_difference"] = mean_absolute_difference

        if utilitarian_welfare > 0:
            mean_utility = utilitarian_welfare / 2.0
            variance = (p1_value**2 + p2_value**2) / 2.0 - mean_utility**2
            variance = max(variance, 0.0)  # Correct for negative variance due to precision
            coefficient_of_variation = (
                np.sqrt(variance) / mean_utility if mean_utility != 0 else 0.0
            )
            jain_fairness_index = 1 / (1 + coefficient_of_variation ** 2)
        else:
            jain_fairness_index = 0.0
        metrics["jain_fairness_index"] = jain_fairness_index

        # Security levels already set to 0.0
        metrics["security_level_player_1"] = 0.0
        metrics["security_level_player_2"] = 0.0

        # Determine walk_type based on final action
        walk_type = None
        if metrics["action_played"] == "WALK":
            walk_type = "Player2_WALK"
        elif metrics["action_played"] == "COUNTEROFFER":
            walk_type = "Final_COUNTEROFFER"
        elif metrics["action_played"] == "ACCEPT":
            walk_type = "Final_ACCEPT"
        metrics["walk_type"] = walk_type

        # Check if on Pareto Frontier
        on_pareto_frontier = False
        for vals in pareto_front:
            if vals["type"] == "outside_offer" and game.current_offer is None:
                on_pareto_frontier = True
                break
            elif vals["type"] == "allocation":
                if (np.array_equal(vals["agent1"], p1_kept) and
                        np.array_equal(vals["agent2"], p2_kept)):
                    on_pareto_frontier = True
                    break
        metrics["on_pareto_frontier"] = on_pareto_frontier

    # Mark game as ended
    game.in_progress = False

    return metrics



def find_allocation_less_than_outside_offer_dp(items, player_valuations, outside_offer, player_num):
    """
    Finds the allocation that yields the highest utility strictly less than the outside_offer.
    Using dynamic programming to find the best allocation.
    """
    num_items = len(items)
    best_utility = -1.0
    best_combo = None

    quantity_ranges = [range(items[i] + 1) for i in range(num_items)]
    
    for combo in product(*quantity_ranges):
        
        total_utility = 0.0
        for i in range(num_items):
            total_utility += player_valuations[i] * combo[i]

        if total_utility < outside_offer and total_utility > best_utility:
            best_utility = total_utility
            best_combo = combo

    if best_combo is None:
        return None
    allocation = {}
    for i in range(num_items):
        allocation[i] = best_combo[i]

    return allocation

In [55]:
import time
import numpy as np
import pandas as pd
import torch
from math import sqrt, prod
from eval.game_data import GameData  # Importing GameData from game_data.py


def run_game(circle: int, games: int, max_rounds: int, date: str, game_title: str, llm_type: str):
    """
    Runs a series of negotiation games for a specific circle, tracking comprehensive metrics.

    Args:
        circle (int): The circle parameter influencing allocation strategies.
        games (int): Number of games to simulate.
        max_rounds (int): Maximum number of rounds per game.
        date (str): Date identifier for result files.
        game_title (str): Title identifier for the game series.
        llm_type (str): Type of LLM agent being used (e.g., "openai").
    """
    # Initialize a list to store all GameData instances
    all_game_data = []

    for i in range(games):
        # --------------------------------------------------------------------
        # 1) Per-Game Setup
        # --------------------------------------------------------------------
        # Rate-limit every 10 games to avoid API overuse
        if (i + 1) % 10 == 0:
            print(f"Game {i + 1} of {games}")
            sleep_duration = 2 * np.random.randint(55, 60)  # Sleep for ~2 minutes
            print(f"Sleeping for {sleep_duration} seconds to respect rate limits.")
            time.sleep(sleep_duration)

    
        # --------------------------------------------------------------------
        # 2) Initialize a Single Negotiation Game
        # --------------------------------------------------------------------
        game = NegotitaionGame(
            player1_agent=llm_agent.LLMAgent(llm_type=llm_type, player_num=0),
            player2_agent=llm_agent.LLMAgent(llm_type=llm_type, player_num=1),
            num_items=5,
            item_value_range=[1, 101],
            gamma=0.9,
            max_rounds=max_rounds,
            circle=circle
        )

        # Compute Pareto frontier for reference
        pareto_front = compute_pareto_frontier(
            game.player_values[0],
            game.player_values[1],
            game.num_items,
            game.items,
            game.outside_offer_values
        )

        # --------------------------------------------------------------------
        # 3) Optional: Find Allocations with Utility < Outside Offer (Circles 5 & 6)
        # --------------------------------------------------------------------
        allocations_less_than_outside_offer = None
        if circle in (5, 6):
            allocations_less_than_outside_offer = []

            # Find allocations where Player 1's utility is less than their outside offer
            allocation_p1 = find_allocation_less_than_outside_offer_dp(
                items=game.items,
                player_valuations=game.player_values[0],
                outside_offer=game.outside_offer_values[0],
                player_num=1
            )
            if allocation_p1:
                allocations_less_than_outside_offer.append({
                    'player': 1,
                    'allocation': list(allocation_p1.values())
                })
            else:
                allocations_less_than_outside_offer.append({
                    'player': 1,
                    'allocation': [0] * game.num_items
                })
                print(f"[INFO] No feasible < outside_offer allocation for Player 1 in Game {i + 1}.")

            # Find allocations where Player 2's utility is less than their outside offer
            allocation_p2 = find_allocation_less_than_outside_offer_dp(
                items=game.items,
                player_valuations=game.player_values[1],
                outside_offer=game.outside_offer_values[1],
                player_num=2
            )
            if allocation_p2:
                allocations_less_than_outside_offer.append({
                    'player': 2,
                    'allocation': list(allocation_p2.values())
                })
            else:
                allocations_less_than_outside_offer.append({
                    'player': 2,
                    'allocation': [0] * game.num_items
                })
                print(f"[INFO] No feasible < outside_offer allocation for Player 2 in Game {i + 1}.")

            print(f"[DEBUG] Game {i + 1} allocations_less_than_outside_offer: {allocations_less_than_outside_offer}")

        print(f"[DEBUG] game.items: {game.items}")
        print(f"[DEBUG] allocations_less_than_outside_offer: {allocations_less_than_outside_offer}")

        game_history = GameHistory(
            agent_1_name="Agent1",
            agent_2_name="Agent2",
            num_items=game.num_items,
            items=torch.tensor(game.items),
            agent_1_values=torch.tensor(game.player_values[0]),
            agent_2_values=torch.tensor(game.player_values[1]),
            agent_1_outside_value=game.outside_offer_values[0],
            agent_2_outside_value=game.outside_offer_values[1]
        )
        game_history.agent_1_offers = []
        game_history.agent_2_offers = []

     
        game_data = GameData(
            circle=circle,
            date=date,
            agent1=f"Agent1_{llm_type}",
            agent2=f"Agent2_{llm_type}"
        )

        print(f"[INFO] Starting Game {i + 1} of {games} for Circle {circle}.")

  
        if i > 0:
            # Not necessary when using GameData, but kept for consistency
            print(f"[DEBUG] Processing Game {i} completed.")

        while game.in_progress:
            # Sleep to simulate thinking time and rate-limit API calls
            sleep_duration = circle + .5  # Adjust based on desired rate-limiting
            print(f"[DEBUG] Sleeping for {sleep_duration} seconds before next step.")
            time.sleep(sleep_duration)

            # Determine current step, round, and player
            current_step = len(game.history[0]) + len(game.history[1]) + 1
            current_round = (current_step - 1) // 2 + 1
            current_player = 1 if current_step % 2 == 1 else 2
            game.current_round = current_round

            print("\n" + "=" * 80)
            print(f"Game {i + 1}, Round {current_round}, Player {current_player}'s turn (Step {current_step})")
            print("=" * 80)

            current_allocation_example = None
            if circle in (5, 6) and allocations_less_than_outside_offer is not None:
                if current_player == 1:
                    current_allocation_example = allocations_less_than_outside_offer[0]['allocation']
                elif current_player == 2:
                    current_allocation_example = allocations_less_than_outside_offer[1]['allocation']

            print(f"[DEBUG] Current allocation example type: {type(current_allocation_example)}")

            game.step(example_offer_less_than_outside_offer_self=current_allocation_example)

            action_played = game.players[current_player - 1].action.upper()

            game_data.add_round_data(
                    prompt=game.players[current_player - 1].current_prompt,
                    response=game.players[current_player - 1].current_response,  # Assuming response includes the agent's textual response
                    action=action_played
                )

            if "WALK" in action_played or "ACCEPT" in action_played:
                game.in_progress = False

        # --------------------------------------------------------------------
        # 12) After the Game Loop Ends, Save GameData
        # --------------------------------------------------------------------
        all_game_data.append(game_data)
        #UNCOMMENT THESE TO SAVE EACH GAME'S DATA SEPERATELY 
        # Optionally, save each game's data immediately
        # Filename can include game number, circle, date, etc.
        #filename = f'game_data_{date}_game_{i + 1}_circle_{circle}.json'
        #game_data.save_to_json(filename)
        #print(f"[INFO] Saved GameData to {filename}.")
        #save to pickle
        #filename_pkl = f'game_data_{date}_game_{i + 1}_circle_{circle}.pkl'
        #with open(filename_pkl, "wb") as pf:
        #pickle.dump(game_data, pf)
        #print(f"[INFO] Saved GameData to {filename_pkl}.")

    print("HERE IS THE DATA")
    all_data = {
        "date": date,
        "circle": circle,
        "all_game_data": [gd.to_dict() for gd in all_game_data]
    }
    all_games_filename = f'all_game_data_{date}_{games}_{game_title}_circle_{circle}.json'
    with open(all_games_filename, "w") as f:
        json.dump(all_data, f, indent=4)
        #json.pickle(all_data, f)
    print(f"[INFO] Saved all GameData to JSON file: {all_games_filename}.")

    #save to pickle optinally
    all_games_filename_pkl = f'all_game_data_{date}_{games}_{game_title}_circle_{circle}.pkl'
    with open(all_games_filename_pkl, "wb") as pf:
        pickle.dump(all_data, pf)
    print(f"[INFO] Saved all GameData as a pickle to {all_games_filename_pkl}.")



## Run Games

In [56]:
for circle in circles:
    print(f"Running game for circle {circle}")
    run_game(circle, games, max_rounds, date, prompt_style, llm_type)


Running game for circle 5
[DEBUG] Game 1 allocations_less_than_outside_offer: [{'player': 1, 'allocation': [0, 3, 4, 7, 3]}, {'player': 2, 'allocation': [0, 0, 2, 0, 0]}]
[DEBUG] game.items: [2 3 5 7 3]
[DEBUG] allocations_less_than_outside_offer: [{'player': 1, 'allocation': [0, 3, 4, 7, 3]}, {'player': 2, 'allocation': [0, 0, 2, 0, 0]}]
[INFO] Starting Game 1 of 1 for Circle 5.
[DEBUG] Sleeping for 5.5 seconds before next step.

Game 1, Round 1, Player 1's turn (Step 1)
[DEBUG] Current allocation example type: <class 'list'>

    You and another agent have to negotiate a division of items between the two of you.
    You are Player 1 and the other agent is Player 2.
    There are 5 types of items, called item 1 through item 5.
    There are 2 units of item 1, 3 units of item 2, 5 units of item 3, 7 units of item 4, 3 units of item 5 to divide.
    Both you and Player 2 have a private value per unit of each item type.
    These values are drawn from a uniform random distribution, rangi